In [ ]:
!pip install -q torch==2.1.0 --index-url https://download.pytorch.org/whl/cu121

In [ ]:
!pip install -q "transformers==4.34.0" "datasets==2.13.0" "peft==0.6.2" "accelerate==0.21.0" "bitsandbytes==0.41.2.post2" "trl==0.4.7" "safetensors>=0.3.1" "gdown==4.6"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 95.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.6 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
from google.colab import userdata

In [ ]:
login(token=userdata.get('niru_hf_read'))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!gdown --id 1mAIToBlfmspeINOianc5mP4JdJAx-D2Q

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1mAIToBlfmspeINOianc5mP4JdJAx-D2Q
To: /content/Validation_Dataset_V1.1.zip
100% 6.12M/6.12M [00:00<00:00, 16.6MB/s]


In [ ]:
!unzip Validation_Dataset_V1.1.zip

Archive:  Validation_Dataset_V1.1.zip
   creating: Validation_Dataset_V1.1/Val_Data/
  inflating: Validation_Dataset_V1.1/Val_Data/Airline Dataset.csv  
  inflating: Validation_Dataset_V1.1/Val_Data/BigMart.csv  
  inflating: Validation_Dataset_V1.1/Val_Data/customer_shopping_data.csv  
  inflating: Validation_Dataset_V1.1/Val_Data/FinalFlipkartData.csv  
  inflating: Validation_Dataset_V1.1/Val_Data/Shoe prices.csv  
  inflating: Validation_Dataset_V1.1/validation_questions.csv  


In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

# model_name = "nirusanan/Mistral_7B_SFT_Finetune_Pandas_Code_Gen_Final"
model_name = "nirusanan/Mistral_7B_DPO_Finetune_Pandas_Code_Gen_Final"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="cuda:0",
        trust_remote_code=True
    )
model.config.use_cache = False

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from transformers import pipeline

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=850)

In [ ]:
import re
import traceback
import pandas as pd

def generate_prompt(task, header_columns):
    prompt = f"""Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
{task}

header columns with sample data:
{header_columns}

### Response:
"""
    return prompt

def format_dataframe(df):
    df_dict = df.head(5).to_dict(orient='list')
    formatted_string = "data = {\n"

    for key, values in df_dict.items():
        formatted_string += "    '{}': {},\n".format(key, values)

    formatted_string += "}"
    return formatted_string


def generate_code(prompt):
    result = pipe(prompt)
    generated_text = result[0]['generated_text']
    res = generated_text.split("### End")[0]

    match = re.search("```(.*?)```", res, re.DOTALL)

    if match:
        python_code = match.group(1).strip()
    else:
        python_code = generated_text

    return python_code


def filter_dataframe(df, query):
    python_code = ""

    try:
      header_column = format_dataframe(df)
      prompt = generate_prompt(query, header_column)
      python_code = generate_code(prompt)

      python_code = "global process\n" + python_code
      exec(python_code)
      results = process(df)

      return python_code, results

    except Exception as e:
      full_traceback = traceback.format_exc()

      error_message = f"An error occurred: {e}\n{full_traceback}"

      final_msg = f"{python_code}\n======Error======\n{error_message}"
      return final_msg, "Error"

In [ ]:
qa_df = pd.read_csv('/content/Validation_Dataset_V1.1/validation_questions.csv')
qa_df.head()

,Dataset,Kaggle Link,License,Question
0,BigMart,https://www.kaggle.com/datasets/uniabhi/bigmar...,CC0 1.0,What are the unique Item_Type categories prese...
1,BigMart,https://www.kaggle.com/datasets/uniabhi/bigmar...,CC0 1.0,How many items fall under each Item_Fat_Conten...
2,BigMart,https://www.kaggle.com/datasets/uniabhi/bigmar...,CC0 1.0,List the item identifier of the top 5 items wi...
3,BigMart,https://www.kaggle.com/datasets/uniabhi/bigmar...,CC0 1.0,Which outlet identifier has the highest averag...
4,BigMart,https://www.kaggle.com/datasets/uniabhi/bigmar...,CC0 1.0,Calculate the total item outlet sales for each...


In [ ]:
qa_df.shape

(50, 4)

In [ ]:
records = []

for index, row in qa_df.iterrows():
    print(index)
    dataset_name = row['Dataset']
    question = row['Question']
    dataset_path = '/content/Validation_Dataset_V1.1/Val_Data/' + dataset_name + '.csv'
    val_df = pd.read_csv(dataset_path)
    code, result = filter_dataframe(val_df, question)

    is_error_string = isinstance(result, str) and result == "Error"
    validation = 0 if is_error_string else 1

    records.append({
        'DatasetName': dataset_name,
        'Question': question,
        'Code': code,
        'Result': result,
        'Validation': validation
    })

results_df = pd.DataFrame(records)
results_df.to_csv('results.csv', index=False)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


0


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

47


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


48


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


49
